This tutorial is based on https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/#comments

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical

2024-01-02 07:49:22.482753: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-02 07:49:22.482810: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-02 07:49:22.484160: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-02 07:49:22.492215: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# import pandas as pd

In [3]:
# df = pd.read_csv("../Zoe2.0_Transformer/medium_articles.csv")
# df = df[["text"]]
# df.head()

In [4]:
# df["text"].iloc[0]

In [5]:
# len(df["text"])

In [6]:
# raw_text = df['text'].iloc[:999].str.cat(sep='. ')
# raw_text[:10]
# len(raw_text)  # full text length is 981028347

# with open("medium_articles.txt", "w") as file:
#     file.write(raw_text)

In [7]:
filename = "anna.txt"
# filename = "medium_articles.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()
raw_text = raw_text.replace("\n", " ")
# raw_text[:1000]

In [8]:
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [9]:
n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)

Total Characters:  1985223
Total Vocab:  56


In [10]:
seq_length = 50
n_chars - seq_length

1985173

In [11]:
seq_length = 100
n_chars - seq_length

1985123

In [12]:
raw_text[:1000]

"chapter 1   happy families are all alike; every unhappy family is unhappy in its own way.  everything was in confusion in the oblonskys' house. the wife had discovered that the husband was carrying on an intrigue with a french girl, who had been a governess in their family, and she had announced to her husband that she could not go on living in the same house with him. this position of affairs had now lasted three days, and not only the husband and wife themselves, but all the members of their family and household, were painfully conscious of it. every person in the house felt that there was no sense in their living together, and that the stray people brought together by chance in any inn had more in common with one another than they, the members of the family and household of the oblonskys. the wife did not leave her own room, the husband had not been at home for three days. the children ran wild all over the house; the english governess quarreled with the housekeeper, and wrote to a

In [13]:
type(raw_text)

str

In [14]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
from tqdm import tqdm
import time

# for i in range(0, n_chars - seq_length, 1):
for i in tqdm(range(0, n_chars - seq_length, 1), desc="Processing", unit="iteration"):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])

n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)

Processing: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1985123/1985123 [00:26<00:00, 74558.69iteration/s]

Total Patterns:  1985123


In [15]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = to_categorical(dataY)

print(y.shape[1])

56


In [16]:
# define the LSTM model

# small size model
# model = Sequential()
# model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
# model.add(Dropout(0.2))
# model.add(Dense(y.shape[1], activation='softmax'))
# model.compile(loss='categorical_crossentropy', optimizer='adam')

# larger size model
model = Sequential()
model.add(LSTM(512, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(512))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

2024-01-02 07:50:03.367206: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-02 07:50:03.368780: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-02 07:50:03.402147: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [17]:
# define the checkpoints
filepath="./epoch_new_1_anna/weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [18]:
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20


2024-01-02 07:50:13.671190: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8906
2024-01-02 07:50:14.598926: I external/local_xla/xla/service/service.cc:168] XLA service 0x7fc87240cf80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-02 07:50:14.598963: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100 80GB PCIe, Compute Capability 8.0
2024-01-02 07:50:14.598973: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (1): NVIDIA A100 80GB PCIe, Compute Capability 8.0
2024-01-02 07:50:14.605853: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1704181814.748753 1255888 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


15509/15509 [==============================] - ETA: 0s - loss: 2.0692
Epoch 1: loss improved from inf to 2.06918, saving model to ./epoch_new_1_anna/weights-improvement-01-2.0692-bigger.hdf5
15509/15509 [==============================] - 260s 16ms/step - loss: 2.0692
Epoch 2/20
    4/15509 [..............................] - ETA: 4:38 - loss: 1.7344

/usr/local/lib/python3.11/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


15507/15509 [============================>.] - ETA: 0s - loss: 1.6211
Epoch 2: loss improved from 2.06918 to 1.62113, saving model to ./epoch_new_1_anna/weights-improvement-02-1.6211-bigger.hdf5
15509/15509 [==============================] - 246s 16ms/step - loss: 1.6211
Epoch 3/20
15508/15509 [============================>.] - ETA: 0s - loss: 1.4901
Epoch 3: loss improved from 1.62113 to 1.49009, saving model to ./epoch_new_1_anna/weights-improvement-03-1.4901-bigger.hdf5
15509/15509 [==============================] - 245s 16ms/step - loss: 1.4901
Epoch 4/20
15507/15509 [============================>.] - ETA: 0s - loss: 1.4183
Epoch 4: loss improved from 1.49009 to 1.41827, saving model to ./epoch_new_1_anna/weights-improvement-04-1.4183-bigger.hdf5
15509/15509 [==============================] - 246s 16ms/step - loss: 1.4183
Epoch 5/20
15509/15509 [==============================] - ETA: 0s - loss: 1.3727
Epoch 5: loss improved from 1.41827 to 1.37266, saving model to ./epoch_new_1_ann